# Mapping Global Datasets: GeoJSON Format

### USGS Magnitude 1.0+ Earthquakes, Past Month

In this project, we’ll download a dataset representing all the earthquakes that have occurred in the world during the previous month. Then we’ll make a map showing the location of these earthquakes and how significant each one was. Because the data is stored in the GeoJSON format, we’ll work with it using the json module. Using Plotly’s scatter_geo() plot, we’ll create visualizations that clearly show the global distribution of earthquakes.

#### Downloading Earthquake Data

Download the data from one of the United States Geological Survey’s earthquake data feeds, at https://earthquake.usgs.gov/earthquakes/feed. Make a folder called eq_data inside the folder where you’re saving this project’s programs. Copy the file eq_1_day_m1.geojson into this new folder. Earthquakes are categorized by their magnitude on the Richter scale. This file includes data for all earthquakes with a magnitude M1 or greater that took place in the last 24 hours (at the time of this writing).

#### Examining GeoJSON Data

When you open eq_1_day_m1.geojson, you’ll see that it’s very dense and hard to read:

![eq_1_day_m1.json](images/eq_1_day_m1.png)

This file is formatted more for machines than humans. But we can see that the file contains some dictionaries, as well as information that we’re interested in, such as earthquake magnitudes and locations.

The json module provides a variety of tools for exploring and working with JSON data. Some of these tools will help us reformat the file so we can look at the raw data more easily before we work with it programmatically.

Let’s start by loading the data and displaying it in a format that’s easier to read. This is a long data file, so instead of printing it, we’ll rewrite the data to a new file. Then we can open that file and scroll back and forth through the data more easily:

In [1]:
import json
from pathlib import Path

# Read data as a string and convert to a Python object
path = Path('eq_data/eq_data_1_day_m1.geojson')
contents = path.read_text(encoding='utf-8', errors='replace')  # Specify encoding and handle errors
all_eq_data = json.loads(contents)

# Create a more readable version of the data file.
path = Path('eq_data/readable_eq_data.geojson')
readable_contents = json.dumps(all_eq_data, indent=4)
path.write_text(readable_contents)

212684

When you look in your eq_data directory and open the file readable_eq_data.json, here’s the first part of what you’ll see:

![readable_eq_data.json](images/readable_eq_data.png)

The first part of the file includes a section with the key "metadata". This tells us when the data file was generated and where we can find the data online. It also gives us a human-readable title and the number of earthquakes included in this file. In this 24-hour period, 152 earthquakes were recorded.

This GeoJSON file has a structure that’s helpful for location-based data. The information is stored in a list associated with the key "features". Because this file contains earthquake data, the data is in list form where every item in the list corresponds to a single earthquake. This structure might look confusing, but it’s quite powerful. It allows geologists to store as much information as they need to in a dictionary about each earthquake, and then stuff all those dictionaries into one big list.

Let’s look at a dictionary representing a single earthquake:

![readable_eq_data_1.json](images/readable_eq_data_1.png)

The key "properties" contains a lot of information about each earthquake. We’re mainly interested in the magnitude of each earthquake, associated with the key "mag". We’re also interested in the "title" of each event, which provides a nice summary of its magnitude and location.

The key "geometry" helps us understand where the earthquake occurred. We’ll need this information to map each event. We can find the longitude and the latitude for each earthquake in a list associated with the key "coordinates".

This file contains way more nesting than we’d use in the code we write, so if it looks confusing, don’t worry: Python will handle most of the complexity. We’ll only be working with one or two nesting levels at a time. We’ll start by pulling out a dictionary for each earthquake that was recorded in the 24-hour time period.

#### Making a List of All Earthquakes

First, we’ll make a list that contains all the information about every earthquake that occurred.

In [2]:
# Examine all earthquakes in the dataset.
all_eq_dicts = all_eq_data['features']
print(len(all_eq_dicts))

152


We take the data associated with the key 'features' in the all_eq_data dictionary, and assign it to all_eq_dicts. We know this file contains records of 152 earthquakes, and the output verifies that we’ve captured all the earthquakes in the file.

#### Extracting Magnitudes

We can loop through the list containing data about each earthquake, and extract any information we want. Let’s pull out the magnitude of each earthquake:

In [3]:
mags = []
for eq_dict in all_eq_data['features']:
    mags.append(eq_dict['properties']['mag']) 

We print the first 10 magnitudes, so we can see whether we’re getting the correct data:

In [4]:
print(mags[:10])

[1.48, 1.68, 2.73, 1.6, 4.9, 1.6, 1.04, 2.95, 2.05, 2]


Next, we’ll pull the location data for each earthquake, and then we can make a map of the earthquakes.

#### Extracting Location Data

The location data for each earthquake is stored under the key "geometry". Inside the geometry dictionary is a "coordinates" key, and the first two values in this list are the longitude and latitude. Here’s how we’ll pull this data:

In [5]:
lons, lats = [], []
for eq_dict in all_eq_data['features']:
    lons.append(eq_dict['geometry']['coordinates'][0])
    lats.append(eq_dict['geometry']['coordinates'][1])

When we print the first 5 longitudes and latitudes, the output shows that we’re pulling the correct data:

In [6]:
print(lons[:5])
print(lats[:5])

[-116.0376663, -116.0351667, -118.3755, -146.0214, 151.2159]
[33.1703339, 33.1656667, 35.5161667, 61.6227, 45.3484]


With this data, we can move on to mapping each earthquake.

#### Building a World Map

Using the information we’ve pulled so far, we can build a simple world map. Although it won’t look presentable yet, we want to make sure the information is displayed correctly before focusing on style and presentation issues. Here’s the initial map:

In [7]:
import plotly.express as px

title = all_eq_data['metadata']['title']
fig = px.scatter_geo(lat=lats, lon=lons, title=title)
fig.show()

Now that we know the information in our dataset is being plotted correctly, we can make a few changes to make the map more meaningful and easier to read.

#### Representing Magnitudes

A map of earthquake activity should show the magnitude of each earthquake. We can also include more data, now that we know the data is being plotted correctly.

In [8]:
# Read data as a string and convert to a Python object
path = Path('eq_data/eq_data_30_day_m1.geojson') # 30 day data
contents = path.read_text(encoding='utf-8', errors='replace')
all_eq_data = json.loads(contents)

mags, lons, lats = [], [], []
for eq_dict in all_eq_data['features']:
    mags.append(eq_dict['properties']['mag'])
    lons.append(eq_dict['geometry']['coordinates'][0])
    lats.append(eq_dict['geometry']['coordinates'][1])
    
title = all_eq_data['metadata']['title']
fig = px.scatter_geo(lat=lats, lon=lons, size=mags, title=title)
fig.show()

We load the file eq_data_30_day_m1.geojson, to include a full 30 days’ worth of earthquake activity. Also, the earthquakes with a higher magnitude will show up as larger points on the map.

Earthquakes usually occur near tectonic plate boundaries, and the longer period of earthquake activity included in this map reveals the exact locations of these boundaries.

This map is better, but it’s still difficult to pick out which points represent the most significant earthquakes. We can improve this further by using color to represent magnitudes as well.

#### Customizing Marker Colors

We can use Plotly’s color scales to customize each marker’s color, according
to the severity of the corresponding earthquake. We’ll also use a different
projection for the base map.

In [9]:
fig = px.scatter_geo(lat=lats, lon=lons, size=mags, title=title,
    color=mags,
    color_continuous_scale='Viridis',
    labels={'color':'Magnitude'},
    projection='natural earth',
)
fig.show()

The color scale shows the severity of individual earthquakes; the most severe earthquakes stand out as light-yellow points, in contrast to many darker points. You can also tell which regions of the world have more significant earthquake activity. Here we use the 'natural earth' projection, which rounds the ends of the map.

#### Adding Hover Text

To finish this map, we’ll add some informative text that appears when you hover over the marker representing an earthquake. In addition to showing the longitude and latitude, which appear by default, we’ll show the magnitude and provide a description of the approximate location as well.

To make this change, we need to pull a little more data from the file:

In [10]:
eq_titles = []
for eq_dict in all_eq_data['features']:
    eq_titles.append(eq_dict['properties']['title'])

fig = px.scatter_geo(lat=lats, lon=lons, size=mags, title=title,
    color=mags,
    color_continuous_scale='Viridis',
    labels={'color':'Magnitude'},
    projection='natural earth',
    hover_name=eq_titles,
)
fig.show()

## Conclusion

We’ve created a visually appealing and meaningful map of global earthquake activity that also illustrates the geological structure of the planet. Plotly offers a wide range of ways you can customize the appearance and behavior of your visualizations. Using Plotly’s many options, you can make charts and maps that show exactly what you want them to.